In [ ]:
from types import SimpleNamespace
import numpy as np
import holoviews as hv
import pandas as pd
hv.notebook_extension()
%opts Scatter (color='b') Path [show_legend=True]
pd.set_option('display.max_columns', None)

def dims(df):
    return SimpleNamespace(phi=hv.Dimension(r'$\phi$', unit='rad'),
                           phi_c=hv.Dimension(r'$\phi_c$', unit='rad'),
                           I=hv.Dimension('$I$', unit=r'meV / $\Phi_0$'),
                           I_c=hv.Dimension('$I_c$', unit=r'meV / $\Phi_0$'),
                           B_x=hv.Dimension('$B_x$', unit='T', values=df['B_x']),
                           T=hv.Dimension('$T$', unit='K', values=df['T']), 
                           L=hv.Dimension('$L$', unit='nm', values=df['L']), 
                           g=hv.Dimension('$g$-factor', values=df['g']), 
                           alpha=hv.Dimension(r'$\alpha$', unit='mev nm', values=df['alpha']), 
                           mu=hv.Dimension(r'$\mu$', unit='meV', values=df['mu']), 
                           a=hv.Dimension('$a$', unit='nm', values=df['a']),
                           orbital=hv.Dimension('orbital', values=df['orbital']),
                           leads=hv.Dimension('leads', values=df['with_leads']),
                           zoom=hv.Dimension('zoom', values=[5, 10, 25, 50, 75, 100, 200]))

def dynamic_map(fname, label=None):
    df = pd.read_hdf(fname)
    d = dims(df)
    gb = df.groupby(['T', 'L', 'with_leads', 'orbital', 'g', 'alpha', 'mu', 'a'])
    gb2 = df.groupby(['B_x', 'T', 'L', 'with_leads', 'orbital', 'g', 'alpha', 'mu', 'a'])

    if label is None:
        label = fname

    def current_curve(T, L, leads, orbital, g, alpha, mu, a, zoom):
        gr = gb.get_group((T, L, leads, orbital, g, alpha, mu, a))
        return hv.Path((gr['B_x'], gr['current_c']), kdims=[d.B_x, d.I_c], label=label)[:, 0:zoom]

    def phase_curve(T, L, leads, orbital, g, alpha, mu, a, zoom):
        gr = gb.get_group((T, L, leads, orbital, g, alpha, mu, a))
        return hv.Path((gr['B_x'], gr['phase_c']), kdims=[d.B_x, d.phi_c], label=label)[:, -np.pi:np.pi]

    def current_phase(B_x, T, L, leads, orbital, g, alpha, mu, a, zoom):
        gr = gb2.get_group((B_x, T, L, leads, orbital, g, alpha, mu, a))
        
        phases = np.array(gr['phases'])[0]
        currents = np.array(gr['currents'])[0]
        
        return (hv.Path((phases, currents), kdims=[d.phi, d.I], label=label) *
                hv.Scatter((gr['phase_c'], gr['current_c'])))[:, -zoom:zoom]

    kdims = [d.T, d.L, d.leads, d.orbital, d.g, d.alpha, d.mu, d.a, d.zoom]

    return (hv.DynamicMap(current_curve, kdims=kdims),
            hv.DynamicMap(phase_curve, kdims=kdims),
            hv.DynamicMap(current_phase, kdims=[d.B_x] + kdims))

# Critical current and phase, $B_x$ vs $I_c$ and $B_x$ vs $\phi_c$

In [ ]:
circular_A_incorrect = dynamic_map('data/all_data_circular_A_in_SM_incorrect_vector_potential.hdf', label='wrong $A$')
circular_A_in_SM = dynamic_map('data/all_data_circular_A_in_SM.hdf', label='$A$ in SM')
circular_A_in_SM_and_SC = dynamic_map('data/all_data_circular_A_in_SM_and_SC.hdf', label='correct $A$ in SM and SC')
square_A_in_SM = dynamic_map('data/all_data_square_A_in_SM.hdf')
square_A_in_SM_and_SC = dynamic_map('data/all_data_square_A_in_SM_and_SC.hdf', r'$A$ only in SM')

In [ ]:
circular_A_in_SM[0] * circular_A_in_SM_and_SC[0] + circular_A_in_SM[1] * circular_A_in_SM_and_SC[1]

In [ ]:
square_A_in_SM[0] * square_A_in_SM_and_SC[0] #+ square_A_in_SM[1] * square_A_in_SM_and_SC[1]

# Current phase relation, $\phi$ vs $I$
Adjust `zoom` slider to change the `y`-range of the plot.

In [ ]:
# hm + currents[key] * hv.HoloMap({B: hv.VLine(B) for B in df['B_x'].unique()}, kdims=[d.B_x])
square_A_in_SM[2] * square_A_in_SM_and_SC[2]

# Gate

In [ ]:
%%opts Image [colorbar=True aspect='square']
df = pd.read_hdf('data/all_gate_data_current_as_function_of_gate_and_B_x__A_in_SM_and_SC.hdf')
d = dims(df)
d.V = hv.Dimension('$V$', unit='meV')
inds = ['gate_size', 'T', 'L', 'with_leads', 'orbital', 'g', 'alpha', 'mu', 'a']
kdims = ['gate_size', d.T, d.L, 'infinite leads', 'orbital', d.g, d.alpha, d.mu, d.a]

gb = df.groupby(inds)
bnds = (df['B_x'].min(), df['Vs'].min(), df['B_x'].max(), df['Vs'].max())
gate_sizes = hv.HoloMap({key: hv.Image(np.rot90(val['current_c'].reshape(100, -1)),
                                       bounds=bnds, kdims=[d.B_x, d.V], vdims=[d.I_c])
                         for key, val in gb}, kdims=kdims)


gate_sizes

# Rotating field

In [ ]:
%%opts Path [aspect='square']
to_deg = lambda col: (col * 180 / np.pi).round(1).astype(int)

df = pd.read_hdf('data/rotation_of_field__L_640_nm__infinite_leads.hdf')
df['theta'] = to_deg(df['theta'])
gb = df.groupby(['theta'])
gb2 = df.groupby(['B', 'theta'])
d = dims(df)
d.B = hv.Dimension(r'$|B|$', unit='T', values=df['B'])
d.theta = hv.Dimension(r'$\theta$', unit='deg')

hm = hv.HoloMap({theta: hv.Path((gr['B'], gr['current_c']), kdims=[d.B, d.I_c]) for theta, gr in gb},
                kdims=[d.theta])
hm

In [ ]:
def current_phase(B, theta, zoom):
    gr = gb2.get_group((B, theta))
    phases = np.array(gr['phases'])[0]
    currents = np.array(gr['currents'])[0]
    return (hv.Path((phases, currents), kdims=[d.phi, d.I]) *
            hv.Scatter((gr['phase_c'], gr['current_c'])))[:, -zoom:zoom]

kdims = [hv.Dimension(r'$\theta$', unit='deg', values=df['theta']), d.zoom]
hv.DynamicMap(current_phase, kdims=[d.B] + kdims)

# B in $y$

In [ ]:
df = pd.read_hdf('test.hdf')
gb = df.groupby(['orbital', 'g', 'alpha', 'mu'])
d = dims(df)
d.B = hv.Dimension(r'$|B|$', unit='T')

In [ ]:
%%opts Path [aspect='square']
hm = hv.HoloMap({key: hv.Path((gr['B_y'], gr['current_c']), kdims=[d.B, d.I_c]) for key, gr in gb},
                kdims=[d.orbital, d.g, d.alpha, d.mu])
hm